In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2)

In [3]:
train_path=r"C:\\Users\\nalla_enejoi3\\OneDrive\\Desktop\\My_Projects\\COVID-19_Radiography_Dataset\\Training\\dataset\\train"
test_path=r"C:\\Users\\nalla_enejoi3\\OneDrive\\Desktop\\My_Projects\\COVID-19_Radiography_Dataset\\Training\\dataset\\test"

train_data= train_datagen.flow_from_directory(train_path,
                                              target_size=(299,299),
                                              batch_size=32,
                                              class_mode='categorical')
test_data = val_datagen.flow_from_directory(test_path,
                                            target_size=(299,299),
                                            batch_size=32,
                                            class_mode='categorical')

Found 6532 images belonging to 4 classes.
Found 1612 images belonging to 4 classes.


In [4]:
print(train_data.class_indices)
print(test_data.class_indices)

{'covid': 0, 'lung Opacity': 1, 'normal': 2, 'viral Pneumonia': 3}
{'covid': 0, 'lung Opacity': 1, 'normal': 2, 'viral Pneumonia': 3}


In [5]:
from tensorflow.keras.applications import Xception

from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
xception = Xception(input_shape=(299,299,3),include_top=False)

for layers in xception.layers:
  layers.trainable = False

x=Flatten()(xception.output)
x=Dense(256,activation='relu')(x)
output=Dense(4,activation='softmax')(x)
xception=Model(xception.input,output)
xception.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 149, 149,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 149, 149,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 147, 147,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 147, 147,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 147, 147,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 147, 147,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 147, 147,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 147, 147,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 147, 147,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 147, 147,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 74, 74,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 74, 74,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 74, 74,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 74, 74,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 74, 74,    │          0 │ add[0][0]       

 Total params: 73,291,564 (279.59 MB)

 Trainable params: 52,430,084 (200.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [7]:
xception.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

my_callbacks = [EarlyStopping(patience=6)]
xception.fit(train_data,epochs=15,validation_data=test_data , steps_per_epoch=len(train_data)//20,
             validation_steps=len(test_data)//20, callbacks=my_callbacks)

Epoch 1/15


c:\Users\nalla_enejoi3\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


10/10 ━━━━━━━━━━━━━━━━━━━━ 94s 8s/step - accuracy: 0.4100 - loss: 23.4114 - val_accuracy: 0.5938 - val_loss: 12.0039
Epoch 2/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 75s 8s/step - accuracy: 0.6211 - loss: 9.5972 - val_accuracy: 0.7969 - val_loss: 4.9204
Epoch 3/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 78s 8s/step - accuracy: 0.7494 - loss: 5.6245 - val_accuracy: 0.6875 - val_loss: 5.5336
Epoch 4/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 76s 8s/step - accuracy: 0.7347 - loss: 3.8033 - val_accuracy: 0.8125 - val_loss: 2.3995
Epoch 5/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 83s 8s/step - accuracy: 0.7939 - loss: 1.8080 - val_accuracy: 0.8281 - val_loss: 1.6885
Epoch 6/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 88s 9s/step - accuracy: 0.8046 - loss: 1.6949 - val_accuracy: 0.7500 - val_loss: 1.6983
Epoch 7/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 94s 10s/step - accuracy: 0.8279 - loss: 1.3447 - val_accuracy: 0.7500 - val_loss: 3.1757
Epoch 8/15
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 8s/step - accuracy: 0.7984 - loss: 1.7784 - val_accuracy: 0.7188 - val_loss: 2.1917


In [8]:
xception.save("xception.h5")